# Classification à partir d'un arbre de décision et de forêts aléatoires

On va utiliser le jeu de données Pima Indian Diabetes disponible [là par exemple](https://www.kaggle.com/uciml/pima-indians-diabetes-database). Le dataset est déjà chargé dans l'environnement.

## Import des librairies

## Import des données

## Visualisation, analyse exploratoire et gestion des éventuels problèmes sur les données

## Préparation des différents datasets pour la modélisation

## Un premier arbre de décision

Créer et entrainer un arbre de décision puis calculer l'accuracy et afficher l'arbre obtenu.

## Un second arbre de décision

Refaire la même chose en modifiant les paramètres de la classe `DecisionTreeClassifier`, notamment `max_depth` et `criterion`.

## Un peu de validation croisée pour affiner les hyperparamètres

Utiliser la fonction adéquate pour tester différentes valeurs des paramètres(`max_depth` et `criterion`).

## Et une petite forêt aléatoire pour finir...

## ...avec validation croisée pour le nombre d'arbres de la forêt

## Un petit dernier : Gradient Boosting

Gradient Boosting est une amélioration des forêts aléatoires. Il en existe d'autres, notamment XGBoost pour Extreme Gradient Boosting qui est un mdèle performant dans de nombreux cas. Juste un petit aperçu...